In [1]:
#Danas scraper > XML output

from urllib.request import urlopen
import certifi
import ssl
import json
import sys
from urllib.error import HTTPError, URLError
import time

BLANK_PAGE_SZ = 6350   # Not Found Web page size
magazine = "Danas"
urlpath  = "https://pretraziva.rs/pregled/danas"
urlpath2 = "https://pretraziva.rs/prikaz/danas"
urlpath3 = "https://pretraziva.rs/download/danas--"

# Make a list of all available issues

# Get a list of all issues from the JavaScript variable "publicationYears"
str1 = urlopen(urlpath, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
htmlstr = str1.read().decode('utf-8')
i1 = htmlstr.index("{")
i2 = htmlstr.index("};")
dictionary = eval(htmlstr[i1:i2+1])

# Print the information

print("\nYears:", flush=True)
for y in dictionary:
    print(y, end=" ", flush=True)

print("\nAll issues by date:", flush=True)
for y in dictionary:
    print(y)
    issues = dictionary[y]
    for i in issues:
        print("  " + i[1].replace(" ","") + y + ".", flush=True)
print()

# Move the text of an issue to a file

# Iterate all pages by year/issue and write the plain text to new files
for y in dictionary:    # years
    
    issues = dictionary[y]

    for i in issues:    # issues
        ymd = y + "-" + i[1][4:6] + "-" + i[1][0:2]
        iurlpath = urlpath2 + "/" + ymd
        print(y, iurlpath, flush=True)
        
        # Iterate pages

        # Number of pages
        iurlpath2 = iurlpath + "/1"
        istr2 = urlopen(iurlpath2, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
        html2 = istr2.read().decode('utf-8')
        i1 = html2.index("var lastPage = ")
        lp = html2[i1+15:i1+19]
        i2 = lp.index(";")
        npg = eval(lp[:i2])
        print(npg, "pages", flush=True)
        
        # Write text to a new file
        ioutpath = ymd + " " + magazine + ".xml"
        print("Writing text to:", ioutpath, flush=True)
        f = open(ioutpath, "w", encoding='utf-8',newline='')
        
        # Add TEI header
        print('Adding header', flush=True)
        with open("TEI header.txt") as header:
            for line in header:
                f.write(line)

        for pg in range(1, npg+1):
            print("  page", pg, flush=True)
            try:
                try:
                    iurlpath3 = urlpath3 + ymd + "--" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
                except:
                    iurlpath3 = urlpath3 + ymd + "--0" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
            except:
                pass
        
        # Add TEI closed tags at the end
        print('Adding footer', flush=True)
        with open("TEI closed tags.txt") as footer:
            for line in footer:
                f.write(line)
    
        f.close()
        
print('Done.', flush=True)


Years:
1998 2000 
All issues by date:
1998
  01.07.1998.
  02.07.1998.
  03.07.1998.
  04.07.1998.
  06.07.1998.
  07.07.1998.
  08.07.1998.
  10.07.1998.
  11.07.1998.
  13.07.1998.
  14.07.1998.
  15.07.1998.
  16.07.1998.
  17.07.1998.
  18.07.1998.
  09.10.1998.
2000
  01.06.2000.
  02.06.2000.
  03.06.2000.
  05.06.2000.
  06.06.2000.
  07.06.2000.
  08.06.2000.
  09.06.2000.
  10.06.2000.
  12.06.2000.
  13.06.2000.
  14.06.2000.
  15.06.2000.
  16.06.2000.
  17.06.2000.
  19.06.2000.
  20.06.2000.
  22.06.2000.
  23.06.2000.
  24.06.2000.
  26.06.2000.
  27.06.2000.
  28.06.2000.
  29.06.2000.
  30.06.2000.

1998 https://pretraziva.rs/prikaz/danas/1998-07-01
24 pages
Writing text to: 1998-07-01 Danas.xml
Adding header
  page 1
  page 2
  page 3
  page 4
  page 5
  page 6
  page 7
  page 8
  page 9
  page 10
  page 11
  page 12
  page 13
  page 14
  page 15
  page 16
  page 17
  page 18
  page 19
  page 20
  page 21
  page 22
  page 23
  page 24
Adding footer
1998 https://pretra

In [2]:
#Scuffed word count 

import os, glob

total = 0
filecount = 0

for filename in glob.glob('*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        filecount = filecount + 1
        data = f.read()
        words = data.split()
        total = total + len(words)
        
total = total - filecount*57       # 57 is the WC in the two TEI files
print(total)

1486874
